In [1]:
# pandas_gbq
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from time import sleep
from tqdm import tqdm
from locale import atof, setlocale, LC_NUMERIC
setlocale(LC_NUMERIC, '')

import undetected_chromedriver as uc
driver = uc.Chrome(use_subprocess=True)

base_url = "https://www.propertyguru.com.sg"
url = "/property-for-sale?market=residential&beds[]=3&listing_type=sale&property_type=H&mintop=2000&MRT_STATIONS[]=NE9&MRT_STATIONS[]=DT12&MRT_STATIONS[]=NE7&MRT_STATIONS[]=NE8&freetext=NE9+Boon+Keng+MRT,+DT12/NE7+Little+India+MRT,+NE8+Farrer+Park+MRT&search=true"

In [2]:
data = {
    "id": [], "title": [], "href": [], "hero_img": [], 
    "address": [], "price": [], "floor_area_sqft": [], "psf": []
}

In [3]:
def generator(url):
    while url:
        driver.get(f"{base_url}{url}")

        html = driver.page_source
        html_soup = BeautifulSoup(html, 'html.parser')

        listings = html_soup.find_all("div", class_="listing-card")

        for listing in listings:
            nav_link = listing.find_all("a", class_="nav-link")
            data["id"].append(int(nav_link[-1].get("data-listing-id", "")))
            data["title"].append(nav_link[-3].text)
            data["href"].append(nav_link[-3]["href"])
            data["hero_img"].append(nav_link[-2].img["content"] if nav_link[-2].img else "")

            data["address"].append(listing.find_all("span", itemprop="streetAddress")[0].text)
            data["price"].append(atof(listing.find_all("span", class_="price")[0].text))
            data["floor_area_sqft"].append(atof(listing.find_all("li", class_="listing-floorarea")[0].text.split()[0]))
            data["psf"].append(atof(listing.find_all("li", class_="listing-floorarea")[1].text.split(u"\xa0")[1]))

        next_button = html_soup.find_all("li", class_="pagination-next")[0].a
        url = next_button["href"] if next_button else ""
        sleep(10)
            
        yield
        
for _ in tqdm(generator(url)): pass

5it [01:16, 15.38s/it]


In [4]:
df = pd.DataFrame(data)
df["date"] = datetime.today()

df.to_gbq(
    destination_table = "leo-gcp-sanbox.hdb_prices.NE789",
    if_exists = "append",
    progress_bar = True
)

print(df.shape)
df.head()

100%|██████████| 1/1 [00:00<00:00, 6364.65it/s]

(86, 9)


,id,title,href,hero_img,address,price,floor_area_sqft,psf,date
0,24231631,10B Bendemeer Light,https://www.propertyguru.com.sg/listing/hdb-fo...,https://sg2-cdn.pgimgs.com/agent/383688/APHO.1...,10B Bendemeer Road,1050000.0,1001.0,1048.95,2022-11-20 15:11:30.966560
1,24229141,10B Bendemeer Light,https://www.propertyguru.com.sg/listing/hdb-fo...,https://sg2-cdn.pgimgs.com/agent/10723581/APHO...,10B Bendemeer Road,1050000.0,1001.0,1048.95,2022-11-20 15:11:30.966560
2,24229139,10B Bendemeer Light,https://www.propertyguru.com.sg/listing/hdb-fo...,https://sg2-cdn.pgimgs.com/agent/11118124/APHO...,10B Bendemeer Road,1050000.0,1001.0,1048.95,2022-11-20 15:11:30.966560
3,24228967,10B Bendemeer Light,https://www.propertyguru.com.sg/listing/hdb-fo...,https://sg2-cdn.pgimgs.com/agent/11616149/APHO...,10B Bendemeer Road,1050000.0,1001.0,1048.95,2022-11-20 15:11:30.966560
4,24229279,10B Bendemeer Light,https://www.propertyguru.com.sg/listing/hdb-fo...,https://sg1-cdn.pgimgs.com/agent/14278118/APHO...,10B Bendemeer Road,1050000.0,1001.0,1048.95,2022-11-20 15:11:30.966560
